In [1]:
import sys
sys.path.append('/Users/tunadorable/local-repos/next-concept-predictor/venv/lib/python3.11/site-packages')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

# regular GPT

In [3]:
b=2
n=5
d=3
v=10

In [4]:
# create sequence & embedding matrix
S = torch.randint(low=0, high=v, size=(b, n))
E = torch.randn(v,d)

# Create a LayerNorm instance
layer_norm = nn.LayerNorm(d)

# Apply LayerNorm to E
E = layer_norm(E)

# Look up the embeddings
X = F.embedding(S, E)

print("S: ", S.shape, S)
print("E: ", E.shape, E)
print("X: ", X.shape, X)

S:  torch.Size([2, 5]) tensor([[0, 3, 6, 2, 6],
        [6, 7, 1, 9, 1]])
E:  torch.Size([10, 3]) tensor([[ 1.4142, -0.7139, -0.7003],
        [-1.3562,  0.3310,  1.0252],
        [-0.8545, -0.5486,  1.4031],
        [-0.8196, -0.5883,  1.4079],
        [-0.1093, -1.1664,  1.2757],
        [-1.0950,  1.3226, -0.2276],
        [ 0.9015,  0.4929, -1.3944],
        [-1.3350,  1.0717,  0.2633],
        [ 1.3191, -1.1010, -0.2181],
        [-1.3979,  0.5137,  0.8842]], grad_fn=<NativeLayerNormBackward0>)
X:  torch.Size([2, 5, 3]) tensor([[[ 1.4142, -0.7139, -0.7003],
         [-0.8196, -0.5883,  1.4079],
         [ 0.9015,  0.4929, -1.3944],
         [-0.8545, -0.5486,  1.4031],
         [ 0.9015,  0.4929, -1.3944]],

        [[ 0.9015,  0.4929, -1.3944],
         [-1.3350,  1.0717,  0.2633],
         [-1.3562,  0.3310,  1.0252],
         [-1.3979,  0.5137,  0.8842],
         [-1.3562,  0.3310,  1.0252]]], grad_fn=<EmbeddingBackward0>)


In [6]:
# Apply layer normalization to X before each residual add
Xf = layer_norm(X)

print("Xf: ", Xf.shape, Xf)

Xf:  torch.Size([2, 5, 3]) tensor([[[ 1.4142, -0.7139, -0.7003],
         [-0.8196, -0.5883,  1.4079],
         [ 0.9015,  0.4929, -1.3944],
         [-0.8545, -0.5486,  1.4031],
         [ 0.9015,  0.4929, -1.3944]],

        [[ 0.9015,  0.4929, -1.3944],
         [-1.3350,  1.0717,  0.2633],
         [-1.3562,  0.3310,  1.0252],
         [-1.3979,  0.5137,  0.8842],
         [-1.3562,  0.3310,  1.0252]]], grad_fn=<NativeLayerNormBackward0>)


In [7]:
Z = torch.matmul(Xf,E.T)

print("Z: ", Z.shape, Z)

Z:  torch.Size([2, 5, 10]) tensor([[[ 3.0000, -2.8721, -1.7994, -1.7250, -0.2152, -2.3333,  1.8994,
          -2.8374,  2.8043, -2.9628],
         [-1.7250,  2.3602,  2.9986,  2.9999,  2.5718, -0.2010, -2.9920,
           0.8344, -0.7406,  2.0884],
         [ 1.8994, -2.4889, -2.9973, -2.9920, -2.4524, -0.0178,  3.0000,
          -1.0423,  0.9506, -2.2398],
         [-1.7994,  2.4158,  3.0000,  2.9985,  2.5233, -0.1092, -2.9973,
           0.9223, -0.8293,  2.1534],
         [ 1.8994, -2.4889, -2.9973, -2.9920, -2.4524, -0.0178,  3.0000,
          -1.0423,  0.9506, -2.2398]],

        [[ 1.8994, -2.4889, -2.9973, -2.9920, -2.4524, -0.0178,  3.0000,
          -1.0423,  0.9506, -2.2398],
         [-2.8374,  2.4351,  0.9223,  0.8344, -0.7682,  2.8192, -1.0423,
           3.0000, -2.9984,  2.6495],
         [-2.8722,  2.9999,  2.4159,  2.3602,  1.0700,  1.6895, -2.4890,
           2.4352, -2.3771,  2.9724],
         [-2.9629,  2.9724,  2.1534,  2.0884,  0.6816,  2.0089, -2.2399,
          

In [8]:
Zn = Z[:,-1]
print("Zn: ", Zn.shape, Zn)

Zn:  torch.Size([2, 10]) tensor([[ 1.8994, -2.4889, -2.9973, -2.9920, -2.4524, -0.0178,  3.0000, -1.0423,
          0.9506, -2.2398],
        [-2.8722,  2.9999,  2.4159,  2.3602,  1.0700,  1.6895, -2.4890,  2.4352,
         -2.3771,  2.9724]], grad_fn=<SelectBackward0>)


### only used for training

In [9]:
# Generate random indices for one-hot positions
# Each index is between 0 and v-1
ideal_indices = torch.randint(low=0, high=v, size=(b,))

# Create the one-hot matrix
Q = torch.nn.functional.one_hot(ideal_indices, num_classes=v).float()

print("Q: ", Q.shape, Q)

Q:  torch.Size([2, 10]) tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])


In [10]:
# cross-entropy loss function includes within it the softmax function
loss = torch.nn.functional.cross_entropy(Zn,Q)
print(loss, loss.shape)

tensor(4.8813, grad_fn=<DivBackward1>) torch.Size([])


### only used for inference

In [11]:
# Apply softmax along the last dimension (v)
P = F.softmax(Zn, dim=-1)
print("P: ", P.shape, P)
print(torch.sum(Zn[0], dim=-1), torch.sum(P[0], dim=-1))

P:  torch.Size([2, 10]) tensor([[0.2151, 0.0027, 0.0016, 0.0016, 0.0028, 0.0316, 0.6465, 0.0114, 0.0833,
         0.0034],
        [0.0007, 0.2467, 0.1376, 0.1301, 0.0358, 0.0665, 0.0010, 0.1403, 0.0011,
         0.2400]], grad_fn=<SoftmaxBackward0>)
tensor(-8.3805, grad_fn=<SumBackward1>) tensor(1.0000, grad_fn=<SumBackward1>)


In [12]:
# finding tokens using greedy decoding
max_probs, indices = torch.max(P, dim=1)
print("max_probs: ", max_probs.shape, max_probs)
print("indices: ", indices.shape, indices)

max_probs:  torch.Size([2]) tensor([0.6465, 0.2467], grad_fn=<MaxBackward0>)
indices:  torch.Size([2]) tensor([6, 1])


In [13]:
# appending tokens to s
# Reshape new tokens to shape (b, 1)
indices = indices.unsqueeze(1)  # Adds a new dimension

# Concatenate the tensors
S_nplus1 = torch.cat((S, indices), dim=1)  # Concatenate along the sequence length dimension
print("S: ", S.shape, S)
print("S_nplus1: ", S_nplus1.shape, S_nplus1)

S_nplus1:  torch.Size([2, 6]) tensor([[0, 3, 6, 2, 6, 6],
        [6, 7, 1, 9, 1, 1]])


# stage 1: implementing cosine norm, sin, & cosine similarity

In [3]:
b=2
n=5
d=3
v=10

In [4]:
# create sequence & embedding matrix
S = torch.randint(low=0, high=v, size=(b, n))
E = torch.randn(v,d)

# cosine norm for E
E = E / torch.norm(E, p=2, dim=1, keepdim=True) 

# Look up the embeddings
X = F.embedding(S, E)

print("S: ", S.shape, S)
print("E: ", E.shape, E)
print("X: ", X.shape, X)

S:  torch.Size([2, 5]) tensor([[8, 8, 9, 1, 8],
        [1, 4, 2, 7, 3]])
E:  torch.Size([10, 3]) tensor([[-0.9611, -0.2614, -0.0892],
        [-0.0176,  0.6797, -0.7333],
        [-0.3649,  0.8837,  0.2932],
        [ 0.8743, -0.4573,  0.1626],
        [ 0.5383,  0.7788,  0.3219],
        [ 0.4626,  0.6339,  0.6198],
        [-0.7114, -0.6084,  0.3519],
        [ 0.1640, -0.6483,  0.7436],
        [ 0.1038,  0.1118,  0.9883],
        [ 0.4164,  0.8637,  0.2838]])
X:  torch.Size([2, 5, 3]) tensor([[[ 0.1038,  0.1118,  0.9883],
         [ 0.1038,  0.1118,  0.9883],
         [ 0.4164,  0.8637,  0.2838],
         [-0.0176,  0.6797, -0.7333],
         [ 0.1038,  0.1118,  0.9883]],

        [[-0.0176,  0.6797, -0.7333],
         [ 0.5383,  0.7788,  0.3219],
         [-0.3649,  0.8837,  0.2932],
         [ 0.1640, -0.6483,  0.7436],
         [ 0.8743, -0.4573,  0.1626]]])


In [5]:
Xi=torch.sin(torch.pi*X)
print("Xi: ", Xi.shape, Xi)

Xi:  torch.Size([2, 5, 3]) tensor([[[ 0.3202,  0.3440,  0.0367],
         [ 0.3202,  0.3440,  0.0367],
         [ 0.9657,  0.4151,  0.7782],
         [-0.0552,  0.8448, -0.7433],
         [ 0.3202,  0.3440,  0.0367]],

        [[-0.0552,  0.8448, -0.7433],
         [ 0.9928,  0.6403,  0.8476],
         [-0.9113,  0.3574,  0.7963],
         [ 0.4926, -0.8935,  0.7213],
         [ 0.3847, -0.9910,  0.4889]]])


In [6]:
# Normalize each vector to have a unit length
norms = torch.norm(Xi, p=2, dim=2, keepdim=True)
Xf = Xi / norms
print("norms: ", norms.shape, norms)
print("Xf: ", Xf.shape, Xf)

norms:  torch.Size([2, 5, 1]) tensor([[[0.4714],
         [0.4714],
         [1.3079],
         [1.1266],
         [0.4714]],

        [[1.1266],
         [1.4539],
         [1.2618],
         [1.2495],
         [1.1701]]])
Xf:  torch.Size([2, 5, 3]) tensor([[[ 0.6793,  0.7297,  0.0779],
         [ 0.6793,  0.7297,  0.0779],
         [ 0.7384,  0.3174,  0.5950],
         [-0.0490,  0.7499, -0.6597],
         [ 0.6793,  0.7297,  0.0779]],

        [[-0.0490,  0.7499, -0.6597],
         [ 0.6828,  0.4404,  0.5830],
         [-0.7222,  0.2832,  0.6311],
         [ 0.3942, -0.7151,  0.5773],
         [ 0.3288, -0.8470,  0.4178]]])


In [7]:
# let's check that it worked just making sure

# Calculate the L2 norm for each row in X_normalized
norms_after_normalization = torch.norm(Xf, p=2, dim=2)

# Check if each norm is approximately equal to 1
# We use a small threshold to account for numerical precision issues
is_normalized = torch.isclose(norms_after_normalization, torch.ones_like(norms_after_normalization), atol=1e-6)

# Print the result
print("Each row is normalized:", is_normalized.all().item())
print(norms_after_normalization)

Each row is normalized: True
tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000]])


In [8]:
W = torch.randn(b,n)
C = torch.matmul(W,Xf)
print("C: ", C.shape, C)

C:  torch.Size([2, 2, 3]) tensor([[[ 0.1036, -0.0548,  0.1386],
         [-1.7745, -0.5268, -1.2985]],

        [[-0.6565,  2.1135, -1.8130],
         [-0.4623, -0.9411,  0.4970]]])


In [9]:
# Normalize each vector to have a unit length & select only final row
C_select = C[-1]
print("C_select: ", C_select.shape, C_select)

C_norms = torch.norm(C_select, p=2, dim=1, keepdim=True)
print("C_norms: ", C_norms.shape, C_norms)

C_prime = C_select / C_norms
print("C_prime: ", C_prime.shape, C_prime)

C_select:  torch.Size([2, 3]) tensor([[-0.6565,  2.1135, -1.8130],
        [-0.4623, -0.9411,  0.4970]])
C_norms:  torch.Size([2, 1]) tensor([[2.8609],
        [1.1603]])
C_prime:  torch.Size([2, 3]) tensor([[-0.2295,  0.7387, -0.6337],
        [-0.3984, -0.8111,  0.4283]])


In [10]:
# Perform cosine similarity, which simplifies down to dot product
# We transpose E to get shape (d, v) and then use torch.matmul for batched matrix multiplication
Y = torch.matmul(C_prime, E.T)
print("E.T: ", E.T.shape, E.T)
print("Y: ", Y.shape, Y)

# let's make sure Y looks good
# Calculating statistics
max_values = torch.max(Y, dim=1).values
min_values = torch.min(Y, dim=1).values
mean_values = torch.mean(Y, dim=1)
median_values = torch.median(Y, dim=1).values

print("Y Max:", max_values)
print("Y Min:", min_values)
print("Y Mean:", mean_values)

E.T:  torch.Size([3, 10]) tensor([[-0.9611, -0.0176, -0.3649,  0.8743,  0.5383,  0.4626, -0.7114,  0.1640,
          0.1038,  0.4164],
        [-0.2614,  0.6797,  0.8837, -0.4573,  0.7788,  0.6339, -0.6084, -0.6483,
          0.1118,  0.8637],
        [-0.0892, -0.7333,  0.2932,  0.1626,  0.3219,  0.6198,  0.3519,  0.7436,
          0.9883,  0.2838]])
Y:  torch.Size([2, 10]) tensor([[ 0.0839,  0.9709,  0.5507, -0.6415,  0.2478, -0.0307, -0.5092, -0.9877,
         -0.5675,  0.3626],
        [ 0.5567, -0.8584, -0.4458,  0.0922, -0.7083, -0.4329,  0.9276,  0.7789,
          0.2913, -0.7449]])
Y Max: tensor([0.9709, 0.9276])
Y Min: tensor([-0.9877, -0.8584])
Y Mean: tensor([-0.0521, -0.0543])


### only used for training

In [11]:
# Generate random indices for one-hot positions
# Each index is between 0 and v-1
ideal_indices = torch.randint(low=0, high=v, size=(b,))

# Create the one-hot matrix
Q = torch.nn.functional.one_hot(ideal_indices, num_classes=v).float()

print("Q: ", Q.shape, Q)

Q:  torch.Size([2, 10]) tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [12]:
# Create a mask where zeros are marked
zero_mask = Q == 0

# Replace zeros with -1
Q[zero_mask] = -1
print("Q_prime: ", Q.shape, Q)

Q_prime:  torch.Size([2, 10]) tensor([[-1., -1., -1., -1., -1.,  1., -1., -1., -1., -1.],
        [ 1., -1., -1., -1., -1., -1., -1., -1., -1., -1.]])


In [13]:
# cross-entropy loss function includes within it the softmax function
loss = torch.nn.functional.cross_entropy(Y,Q)
print(loss, loss.shape)

tensor(-20.5206) torch.Size([])


### only used for inference

In [19]:
# Apply softmax along the last dimension (v)
P = F.softmax(Y, dim=-1)
print("P: ", P.shape, P)
print(torch.sum(Y[0], dim=-1), torch.sum(P[0], dim=-1))

P:  torch.Size([2, 10]) tensor([[0.0730, 0.0447, 0.0368, 0.0468, 0.2062, 0.1478, 0.1027, 0.2300, 0.0788,
         0.0334],
        [0.0564, 0.0384, 0.0354, 0.0585, 0.1769, 0.1807, 0.1176, 0.2029, 0.0992,
         0.0340]])
tensor(-1.1254) tensor(1.)


In [20]:
# finding tokens using greedy decoding
max_probs, indices = torch.max(P, dim=1)
print("max probs: ", max_probs.shape, max_probs)
print("indices: ", indices.shape, indices)

max probs:  torch.Size([2]) tensor([0.2300, 0.2029])
indices:  torch.Size([2]) tensor([7, 7])


In [21]:
# appending tokens to s
# Reshape new tokens to shape (b, 1)
indices = indices.unsqueeze(1)  # Adds a new dimension

# Concatenate the tensors
S_nplus1 = torch.cat((S, indices), dim=1)  # Concatenate along the sequence length dimension
print("S: ", S.shape, S)
print("S_nplus1: ", S_nplus1.shape, S_nplus1)

S:  torch.Size([2, 5]) tensor([[9, 6, 3, 8, 9],
        [3, 5, 1, 5, 9]])
S_nplus1:  torch.Size([2, 6]) tensor([[9, 6, 3, 8, 9, 7],
        [3, 5, 1, 5, 9, 7]])


# stage 2: implementing gamma-neighborhood selection and in-sequence concept embedding vectors

In [3]:
b=2
n=5
d=3
v=10

In [4]:
# create sequence & embedding matrix
S = torch.randint(low=0, high=v, size=(b, n))
E = torch.randn(v,d)

# cosine norm for E
E = E / torch.norm(E, p=2, dim=1, keepdim=True) 

# Look up the embeddings
X = F.embedding(S, E)

print("S: ", S.shape, S)
print("E: ", E.shape, E)
print("X: ", X.shape, X)

S:  torch.Size([2, 5]) tensor([[4, 8, 7, 4, 0],
        [5, 9, 8, 7, 1]])
E:  torch.Size([10, 3]) tensor([[-0.0038,  0.9374, -0.3481],
        [-0.2898,  0.8896,  0.3529],
        [ 0.3230, -0.9239,  0.2051],
        [-0.0869,  0.6198, -0.7799],
        [-0.8146, -0.3411,  0.4692],
        [-0.2261,  0.0461,  0.9730],
        [ 0.3822,  0.0012, -0.9241],
        [ 0.8660, -0.1081, -0.4882],
        [-0.1144, -0.9439, -0.3097],
        [ 0.6993,  0.0981,  0.7080]])
X:  torch.Size([2, 5, 3]) tensor([[[-0.8146, -0.3411,  0.4692],
         [-0.1144, -0.9439, -0.3097],
         [ 0.8660, -0.1081, -0.4882],
         [-0.8146, -0.3411,  0.4692],
         [-0.0038,  0.9374, -0.3481]],

        [[-0.2261,  0.0461,  0.9730],
         [ 0.6993,  0.0981,  0.7080],
         [-0.1144, -0.9439, -0.3097],
         [ 0.8660, -0.1081, -0.4882],
         [-0.2898,  0.8896,  0.3529]]])


In [5]:
Xi=torch.sin(torch.pi*X)
print("Xi: ", Xi.shape, Xi)

Xi:  torch.Size([2, 5, 3]) tensor([[[-0.5501, -0.8779,  0.9953],
         [-0.3516, -0.1752, -0.8265],
         [ 0.4086, -0.3332, -0.9993],
         [-0.5501, -0.8779,  0.9953],
         [-0.0119,  0.1953, -0.8883]],

        [[-0.6521,  0.1445,  0.0847],
         [ 0.8103,  0.3033,  0.7939],
         [-0.3516, -0.1752, -0.8265],
         [ 0.4086, -0.3332, -0.9993],
         [-0.7897,  0.3398,  0.8952]]])


In [6]:
# Normalize each vector to have a unit length
norms = torch.norm(Xi, p=2, dim=2, keepdim=True)
Xf = Xi / norms
print("norms: ", norms.shape, norms)
print("Xf: ", Xf.shape, Xf)

norms:  torch.Size([2, 5, 1]) tensor([[[1.4367],
         [0.9151],
         [1.1299],
         [1.4367],
         [0.9096]],

        [[0.6733],
         [1.1742],
         [0.9151],
         [1.1299],
         [1.2411]]])
Xf:  torch.Size([2, 5, 3]) tensor([[[-0.3829, -0.6111,  0.6928],
         [-0.3842, -0.1915, -0.9032],
         [ 0.3617, -0.2949, -0.8844],
         [-0.3829, -0.6111,  0.6928],
         [-0.0131,  0.2147, -0.9766]],

        [[-0.9686,  0.2146,  0.1258],
         [ 0.6900,  0.2583,  0.6761],
         [-0.3842, -0.1915, -0.9032],
         [ 0.3617, -0.2949, -0.8844],
         [-0.6363,  0.2738,  0.7212]]])


In [7]:
W = torch.randn(b,n)
C = torch.matmul(W,Xf)
print("C: ", C.shape, C)

C:  torch.Size([2, 2, 3]) tensor([[[-0.1230,  0.7641,  1.2563],
         [ 0.4142, -0.6066,  0.5286]],

        [[ 1.0814, -0.6043, -0.5363],
         [ 0.0151, -0.4517, -1.7105]]])


In [8]:
# Normalize each vector to have a unit length & select only final row
C_last = C[-1]
C_last_norms = torch.norm(C_last, p=2, dim=1, keepdim=True)
C_prime = C_last / C_last_norms

print("C_last:", C_last.shape, C_last)
print("C_last_norms:", C_last_norms.shape, C_last_norms)
print("C_prime", C_prime.shape, C_prime)

C_last: torch.Size([2, 3]) tensor([[ 1.0814, -0.6043, -0.5363],
        [ 0.0151, -0.4517, -1.7105]])
C_last_norms: torch.Size([2, 1]) tensor([[1.3499],
        [1.7692]])
C_prime torch.Size([2, 3]) tensor([[ 0.8011, -0.4476, -0.3973],
        [ 0.0086, -0.2553, -0.9668]])


In [9]:
# Perform cosine similarity, which simplifies down to dot product
# We transpose E to get shape (d, v) and then use torch.matmul for batched matrix multiplication
Y = torch.matmul(C_prime, E.T)
print("E.T: ", E.T.shape, E.T)
print("Y: ", Y.shape, Y)

# let's make sure Y looks good
# Calculating statistics
max_values = torch.max(Y, dim=1).values
min_values = torch.min(Y, dim=1).values
mean_values = torch.mean(Y, dim=1)
median_values = torch.median(Y, dim=1).values

# Printing the results
print("Y Max:", max_values)
print("Y Min:", min_values)
print("Y Mean:", mean_values)

E.T:  torch.Size([3, 10]) tensor([[-0.0038, -0.2898,  0.3230, -0.0869, -0.8146, -0.2261,  0.3822,  0.8660,
         -0.1144,  0.6993],
        [ 0.9374,  0.8896, -0.9239,  0.6198, -0.3411,  0.0461,  0.0012, -0.1081,
         -0.9439,  0.0981],
        [-0.3481,  0.3529,  0.2051, -0.7799,  0.4692,  0.9730, -0.9241, -0.4882,
         -0.3097,  0.7080]])
Y:  torch.Size([2, 10]) tensor([[-0.2844, -0.7706,  0.5909, -0.0372, -0.6863, -0.5884,  0.6728,  0.9361,
          0.4539,  0.2350],
        [ 0.0972, -0.5709,  0.0404,  0.5951, -0.3735, -0.9544,  0.8964,  0.5070,
          0.5394, -0.7036]])
Y Max: tensor([0.9361, 0.8964])
Y Min: tensor([-0.7706, -0.9544])
Y Mean: tensor([0.0522, 0.0073])


### the conditional parts

In [10]:
# finding closest tokens to the raw output & their similarity scores
G, H = torch.max(Y, dim=1)
print("G: ", G.shape, G)
print("H: ", H.shape, H)

G:  torch.Size([2]) tensor([0.9361, 0.8964])
H:  torch.Size([2]) tensor([7, 6])


In [11]:
gamma = 0.91 # $-1\leq \gamma < 1$

In [12]:
A = (G > gamma).float()
print("A: ", A.dtype, A.shape, A)

A:  torch.float32 torch.Size([2]) tensor([1., 0.])


In [13]:
A_unsqueeze = A.unsqueeze(1)
print("A_unsqueeze: ", A_unsqueeze.shape, A_unsqueeze)
A_expand = A_unsqueeze.expand(-1, v)
print("A_expand: ", A_expand.shape, A_expand)

A_unsqueeze:  torch.Size([2, 1]) tensor([[1.],
        [0.]])
A_expand:  torch.Size([2, 10]) tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [14]:
I = torch.ones(b,v).float()
print("I: ", I.dtype, I.shape, I)
A_prime_expand = I-A_expand
print("A_prime_expand: ", A_prime_expand.dtype, A_prime_expand.shape, A_prime_expand)

I:  torch.float32 torch.Size([2, 10]) tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
A_prime_expand:  torch.float32 torch.Size([2, 10]) tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])


### only used for trainnig

In [15]:
# Generate random indices for one-hot positions
# Each index is between 0 and v-1
ideal_indices = torch.randint(low=0, high=v, size=(b,))
print("ideal_indices: ", ideal_indices.shape, ideal_indices)

# Create the one-hot matrix
Q = torch.nn.functional.one_hot(ideal_indices, num_classes=v).float()

print("Q: ", Q.dtype, Q.shape, Q)

ideal_indices:  torch.Size([2]) tensor([9, 7])
Q:  torch.float32 torch.Size([2, 10]) tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])


In [16]:
# Create a mask where zeros are marked
zero_mask_Q = Q == 0

# Replace zeros with -1
Q[zero_mask_Q] = -1
print("Q_prime: ", Q.shape, Q)

Q_prime:  torch.Size([2, 10]) tensor([[-1., -1., -1., -1., -1., -1., -1., -1., -1.,  1.],
        [-1., -1., -1., -1., -1., -1., -1.,  1., -1., -1.]])


In [17]:
Q_ideal = Q*A_expand
print("Q_ideal: ", Q_ideal.shape, Q_ideal)

Q_ideal:  torch.Size([2, 10]) tensor([[-1., -1., -1., -1., -1., -1., -1., -1., -1.,  1.],
        [-0., -0., -0., -0., -0., -0., -0.,  0., -0., -0.]])


In [18]:
Y_concepts = Y*A_prime_expand
print("Y_concepts: ", Y_concepts.shape, Y_concepts)

Y_concepts:  torch.Size([2, 10]) tensor([[-0.0000, -0.0000,  0.0000, -0.0000, -0.0000, -0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0972, -0.5709,  0.0404,  0.5951, -0.3735, -0.9544,  0.8964,  0.5070,
          0.5394, -0.7036]])


In [19]:
Y_train = Y_concepts + Q_ideal
print("Y_train: ", Y_train.shape, Y_train)

Y_train:  torch.Size([2, 10]) tensor([[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000,  1.0000],
        [ 0.0972, -0.5709,  0.0404,  0.5951, -0.3735, -0.9544,  0.8964,  0.5070,
          0.5394, -0.7036]])


### only used for inference

In [20]:
# Apply softmax along the last dimension (v)
P = F.softmax(Y, dim=-1)
print("P: ", P.shape, P)
print(torch.sum(Y[0], dim=-1), torch.sum(P[0], dim=-1))

P:  torch.Size([2, 10]) tensor([[0.0607, 0.0373, 0.1457, 0.0778, 0.0406, 0.0448, 0.1581, 0.2058, 0.1271,
         0.1021],
        [0.0925, 0.0474, 0.0874, 0.1522, 0.0578, 0.0323, 0.2057, 0.1393, 0.1439,
         0.0415]])
tensor(0.5220) tensor(1.)


In [21]:
S_nplus1 = torch.max(P, dim=1).indices
print("S_nplus1: ", S_nplus1.shape, S_nplus1)

S_nplus1:  torch.Size([2]) tensor([7, 6])


In [23]:
# Create a mask where zeros are marked
zero_mask_A = A == 0

# Replace zeros with -1
S_nplus1[zero_mask_A] = -1

print("S_nplus1_prime: ", S_nplus1.shape, S_nplus1)

S_nplus1_prime:  torch.Size([2]) tensor([ 7, -1])


In [25]:
print("S_old: ", S.shape, S)
S_nplus1_unsqueeze = S_nplus1.unsqueeze(dim=1)
S = torch.concat((S,S_nplus1_unsqueeze), dim=1)
print("S_new: ", S.shape, S)

S_old:  torch.Size([2, 5]) tensor([[4, 8, 7, 4, 0],
        [5, 9, 8, 7, 1]])
S_new:  torch.Size([2, 6]) tensor([[ 4,  8,  7,  4,  0,  7],
        [ 5,  9,  8,  7,  1, -1]])


In [26]:
epsilon = torch.zeros(1,d)
print("epsilon: ", epsilon.shape, epsilon)
E_prime = torch.cat((E,epsilon), dim=0)
print("E_prime: ", E_prime.shape, E_prime)

epsilon:  torch.Size([1, 3]) tensor([[0., 0., 0.]])
E_prime:  torch.Size([11, 3]) tensor([[-0.0038,  0.9374, -0.3481],
        [-0.2898,  0.8896,  0.3529],
        [ 0.3230, -0.9239,  0.2051],
        [-0.0869,  0.6198, -0.7799],
        [-0.8146, -0.3411,  0.4692],
        [-0.2261,  0.0461,  0.9730],
        [ 0.3822,  0.0012, -0.9241],
        [ 0.8660, -0.1081, -0.4882],
        [-0.1144, -0.9439, -0.3097],
        [ 0.6993,  0.0981,  0.7080],
        [ 0.0000,  0.0000,  0.0000]])


In [28]:
Z_token = F.embedding(S_nplus1,E_prime)
print("Z_token: ", Z_token.shape, Z_token)

IndexError: index out of range in self

In [47]:
C_prime_concept = C_prime*A_prime_expand[:,0:d]
print("C_prime_concept: ", C_prime_concept.shape, C_prime_concept)

C_prime_concept:  torch.Size([2, 3]) tensor([[-0.4552, -0.2610, -0.8513],
        [ 0.0000, -0.0000, -0.0000]])


In [48]:
Z_prime = Z + C_prime_concept
print("Z_prime: ", Z_prime.shape, Z_prime)

Z_prime:  torch.Size([2, 3]) tensor([[-0.4552, -0.2610, -0.8513],
        [ 0.7900, -0.5789, -0.2020]])


In [54]:
print("X: ", X.shape, X)
X_new = torch.cat((X,Z_prime.unsqueeze(1)), dim=1)
print("X_new: ", X_new.shape, X_new)

X:  torch.Size([2, 5, 3]) tensor([[[ 0.1426,  0.6498, -0.7466],
         [-0.8485,  0.5243,  0.0723],
         [-0.9660,  0.1084, -0.2348],
         [ 0.1370,  0.5486, -0.8248],
         [-0.9660,  0.1084, -0.2348]],

        [[ 0.7900, -0.5789, -0.2020],
         [-0.8571, -0.3588, -0.3697],
         [-0.8571, -0.3588, -0.3697],
         [-0.8571, -0.3588, -0.3697],
         [-0.8485,  0.5243,  0.0723]]])
X_new:  torch.Size([2, 6, 3]) tensor([[[ 0.1426,  0.6498, -0.7466],
         [-0.8485,  0.5243,  0.0723],
         [-0.9660,  0.1084, -0.2348],
         [ 0.1370,  0.5486, -0.8248],
         [-0.9660,  0.1084, -0.2348],
         [-0.4552, -0.2610, -0.8513]],

        [[ 0.7900, -0.5789, -0.2020],
         [-0.8571, -0.3588, -0.3697],
         [-0.8571, -0.3588, -0.3697],
         [-0.8571, -0.3588, -0.3697],
         [-0.8485,  0.5243,  0.0723],
         [ 0.7900, -0.5789, -0.2020]]])


## change in gamma over time

In [103]:
gamma_00 = -1
gamma_f0 = -1
gamma_0f = 0.8
gamma_ff = -1

In [104]:
n = 6 # sequence length
m = 9 # num of epochs

In [105]:
delta_gamma_m = (gamma_0f - gamma_00)/m
print(delta_gamma_m)

0.2


In [117]:
for i in range(m):
    gamma_i = gamma_00 + i*delta_gamma_m
    print(f"This epoch, to become a token a concept needs cos similarity at least gamma_0,{i} = {gamma_i:.2f}")
    
    delta_gamma_n = (gamma_i - gamma_ff)/n
    for j in range(2*n):
        
        print(f"\tAt the {j+1}'th token in this sequence, gamma_{j},{i} = {max(-1, gamma_i - j*delta_gamma_n):.2f}")

This epoch, to become a token a concept needs cos similarity at least gamma_0,0 = -1.00
	At the 1'th token in this sequence, gamma_0,0 = -1.00
	At the 2'th token in this sequence, gamma_1,0 = -1.00
	At the 3'th token in this sequence, gamma_2,0 = -1.00
	At the 4'th token in this sequence, gamma_3,0 = -1.00
	At the 5'th token in this sequence, gamma_4,0 = -1.00
	At the 6'th token in this sequence, gamma_5,0 = -1.00
	At the 7'th token in this sequence, gamma_6,0 = -1.00
	At the 8'th token in this sequence, gamma_7,0 = -1.00
	At the 9'th token in this sequence, gamma_8,0 = -1.00
	At the 10'th token in this sequence, gamma_9,0 = -1.00
	At the 11'th token in this sequence, gamma_10,0 = -1.00
	At the 12'th token in this sequence, gamma_11,0 = -1.00
This epoch, to become a token a concept needs cos similarity at least gamma_0,1 = -0.80
	At the 1'th token in this sequence, gamma_0,1 = -0.80
	At the 2'th token in this sequence, gamma_1,1 = -0.83
	At the 3'th token in this sequence, gamma_2,1 = 